Modèle Stat ARIMA... (SARIMA)

librairie : statmodel (library, package ARIMA)


Model LLM, Ollama, TimeCHATGPT...

à compléter et battre le BTC tout seul est quasiment impossible 

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, LSTM
from tensorflow.keras.activations import linear, relu, sigmoid

from modules.utils import feature_engineering_last

from sklearn.model_selection import train_test_split

In [2]:
X = pd.read_csv(
    "/Users/elouan/Repo Github ElouanBahri/Predicting_crypto_prices/Historical Prices for BTCUSDT"
)

In [3]:
# YEARS = [2019,2021,2022,2023,2024,2025]

# Data = filter_data_by_year_month(X, YEARS)

df = feature_engineering_last(X)

In [ ]:
# Check for NaN values in the entire DataFrame
print(df.isnull().sum())

In [4]:
df.head(10)

,open,high,low,close,volume,number_of_trades,return,RSI,upper_band,middle_band,lower_band,MA_7,MA_25,MA_50,MA_100,MACD,MACD_signal,MACD_hist
100,0.004070,0.004106,0.004155,0.003988,0.023472,0.005843,-0.002531,0.707136,0.004013,0.003468,0.002873,0.003654,0.003129,0.002991,0.002911,0.508652,0.512247,0.450707
101,0.003985,0.003938,0.003963,0.003891,0.009838,0.004210,-0.002686,0.658770,0.004063,0.003534,0.002954,0.003742,0.003170,0.003005,0.002918,0.508802,0.512939,0.449665
102,0.003889,0.003824,0.003947,0.003825,0.009466,0.003744,-0.001858,0.626677,0.004072,0.003597,0.003071,0.003822,0.003207,0.003016,0.002926,0.508672,0.513464,0.448328
103,0.003845,0.003851,0.004013,0.003897,0.009093,0.003796,0.002027,0.646516,0.004060,0.003669,0.003228,0.003894,0.003253,0.003029,0.002933,0.508675,0.513884,0.447496
104,0.003901,0.003937,0.004042,0.003881,0.008321,0.003099,-0.000458,0.638055,0.004029,0.003729,0.003381,0.003859,0.003298,0.003042,0.002941,0.508557,0.514193,0.446614
105,0.003875,0.003858,0.004041,0.003917,0.003850,0.002126,0.000995,0.648598,0.003938,0.003795,0.003606,0.003851,0.003339,0.003054,0.002949,0.508478,0.514423,0.445977
106,0.003920,0.003882,0.004103,0.003928,0.003084,0.002301,0.000320,0.652116,0.003795,0.003848,0.003861,0.003830,0.003379,0.003067,0.002957,0.508368,0.514582,0.445414
107,0.003926,0.003844,0.003967,0.003822,0.007355,0.002717,-0.002982,0.591185,0.003737,0.003818,0.003859,0.003806,0.003410,0.003078,0.002964,0.507940,0.514612,0.444387
108,0.003819,0.003781,0.003988,0.003839,0.005603,0.002200,0.000485,0.597597,0.003728,0.003805,0.003843,0.003798,0.003439,0.003090,0.002971,0.507577,0.514554,0.443685
109,0.003831,0.003862,0.004017,0.003943,0.004234,0.002294,0.002923,0.634965,0.003668,0.003791,0.003877,0.003815,0.003466,0.003103,0.002979,0.507490,0.514488,0.443621


In [14]:
# df = df.replace([np.inf, -np.inf], np.nan)  # Replace inf with NaN
# df = df.fillna(df.mean())

X = df.drop(columns=["return"]).values  # Features
y = df["return"].values  # Target variable


# Reshape the data into sequences (timesteps)
timesteps = 10  # Number of timesteps for the RNN
X_sequences = []
y_sequences = []

for i in range(len(X) - timesteps):
    X_sequences.append(X[i : i + timesteps])
    y_sequences.append(y[i + timesteps])

X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

split_index = int(len(X_sequences) * 0.8)  # 80% train, 20% test
X_train, X_test = X_sequences[:split_index], X_sequences[split_index:]
y_train, y_test = y_sequences[:split_index], y_sequences[split_index:]

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

Train shape: (170836, 10, 17), Test shape: (42710, 10, 17)


In [15]:
X_train.shape

(170836, 10, 17)

In [16]:
y_train.shape

(170836,)

In [8]:
print(np.isnan(X).any(), np.isnan(y).any())  # Check for NaN
print(np.isinf(X).any(), np.isinf(y).any())

False False
False False


In [ ]:
# dataset = tf.data.Dataset.from_tensor_slices((X1, y))
# # Calculate sizes for train and validation sets
# dataset_size = len(X1)  # Total number of samples in your dataset
# train_size = int(0.8 * dataset_size)

# # Shuffle the dataset if needed
# dataset = dataset.shuffle(buffer_size=dataset_size)

# # Split the dataset
# train_dataset = dataset.take(train_size)       # Take the first 80%
# val_dataset = dataset.skip(train_size)         # Skip the first 80%, take the remaining 20%

# # Batch both datasets if needed
# batch_size = 32
# train_dataset = train_dataset.batch(batch_size)
# val_dataset = val_dataset.batch(batch_size)

In [17]:
tf.random.set_seed(1234)  # for consistent results

# Simplified GRU Model
model1 = Sequential(
    [
        # Single GRU Layer
        GRU(64, input_shape=(10, X_sequences.shape[2]), name="GRU_Layer"),
        Dropout(0.2, name="Dropout_Layer"),  # Regularization
        # Fully Connected Dense Layer
        Dense(32, activation="relu", name="Dense_Layer"),  # Hidden dense layer
        # Output Layer for Binary Classification
        Dense(1, activation="sigmoid", name="Output_Layer"),  # Predict binary output
    ]
)

/Users/elouan/anaconda3/envs/Predicting_crypto/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [18]:
tf.random.set_seed(1234)  # for consistent results

model2 = Sequential(
    [
        LSTM(64, return_sequences=True, input_shape=(10, X_sequences.shape[2])),
        Dropout(0.2),
        LSTM(32, return_sequences=False),
        Dropout(0.2),
        Dense(16, activation="relu"),
        Dense(1, activation="linear"),  # Output = return prediction
    ]
)

In [19]:
model2.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 10, 64)         │        20,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,953 (132.63 KB)

 Trainable params: 33,953 (132.63 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# model = tf.keras.models.load_model("../models/")

In [ ]:
# Compile the model
model2.compile(
    loss=tf.keras.losses.MeanAbsoluteError(),  # For regression
    # loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),  # Binary classification loss
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Adam optimizer
    metrics=["mae"],  # Track accuracy
)

# Learning rate reduction callback
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,  # Reduce LR by 90% instead of 50%
    patience=3,
    min_lr=1e-6,  # Set a minimum learning rate
)

# Model checkpoint callback
checkpoint = ModelCheckpoint(
    filepath="../models/RNN_LTSM_1.keras",  # Save the model to this path
    # monitor='val_accuracy',                 # Monitor validation accuracy
    # mode='max',
    monitor="val_loss",  # Track loss for regression
    mode="min",  # Lower loss is better                             # Save when accuracy is maximized
    save_best_only=True,  # Save only if the model improves
    verbose=1,  # Display a message when saving
)
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Train the model
history = model2.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=100,  # Increase epochs for better training
    batch_size=32,
    callbacks=[reduce_lr, checkpoint, early_stop],  # Add early stopping
)

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(X_train.reshape(X_train.shape[0], -1), y_train)
print(xgb.score(X_test.reshape(X_test.shape[0], -1), y_test))